In [1]:
!pip install transformers  datasets gradio accelerate -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from datasets import Dataset
from datasets import load_metric

In [49]:
df = pd.read_csv('/content/darija_reviews_cleaned.csv')
df.head()

,cleaned_review,rating,sentiment
0,attention smart mais pas android,4.0,positive
1,excellent qualite prix,5.0,positive
2,mzyanz,5.0,positive
3,koooooooooooooo,1.0,negative
4,ma 3jebatnich,1.0,negative


In [50]:

df['rating'] =df['rating'].astype(str)
df = df.groupby('rating').head(300)
df['rating'].value_counts()


4.0    300
5.0    300
3.0    289
1.0    204
2.0     71
Name: rating, dtype: int64

In [51]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [52]:
train_data['label'] = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
test_data['label'] = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
train_data.drop(columns=['rating','sentiment'],inplace=True)
test_data.drop(columns=['rating','sentiment'],inplace=True)

In [37]:
test_data

,cleaned_review,label
1198,casque zwin dyal gaming li bgha ysm3 music bo7...,4.0
101,bon article,5.0
1216,battery makichedch mezyan kifma dayrin f descr...,3.0
930,mouse mzyana wlkin lkabli dyalha 5yb w9sir,3.0
1183,mzyan walakin formatih hit kiji 3amr b bzf dya...,4.0
...,...,...
1517,mnsb shkr,4.0
136,nqS ltmn,4.0
54,tmm,5.0
1511,they arent much of a sanrio character,4.0


In [53]:
train_ds = Dataset.from_pandas(train_data.dropna())
valid_ds = Dataset.from_pandas(test_data.dropna())

In [54]:
accuracy = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [55]:
tokenizer = AutoTokenizer.from_pretrained('SI2M-Lab/DarijaBERT-arabizi')  # Utilisez le tokenizer correspondant au modèle BERT que vous utilisez


In [56]:
train_ds[0]['cleaned_review']

'je detest fiha 450g o odrya tat9ata3 mika dylha nulll sara7a'

In [57]:
def preprocess_function(examples):
    return tokenizer(examples["cleaned_review"], truncation=True, max_length=384)

train_ds = train_ds.map(preprocess_function, batched=True)
valid_ds = valid_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/912 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [58]:
valid_ds

Dataset({
    features: ['cleaned_review', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 232
})

In [43]:
train_ds = train_ds.remove_columns(['cleaned_review','__index_level_0__'])
valid_ds = valid_ds.remove_columns(['cleaned_review','__index_level_0__'])

In [59]:
train_ds

Dataset({
    features: ['cleaned_review', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 912
})

In [60]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [61]:
from transformers import AutoModelForSequenceClassification

pretrained_model = AutoModelForSequenceClassification.from_pretrained('SI2M-Lab/DarijaBERT-arabizi', num_labels=5)
#pretrained_model.layers[-1].kernel_regularizer = tf.keras.regularizers.l2(0.1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT-arabizi and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
pretrained_model.compile()

In [62]:
from transformers import TrainingArguments  , Trainer
import numpy as np

import torch._dynamo
torch._dynamo.config.suppress_errors = True

training_args = TrainingArguments(
    output_dir="maghrebvibe-bert-train",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",

)

trainer = Trainer(
    model=pretrained_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.556087,0.780172
2,No log,0.479366,0.784483
3,No log,0.406869,0.857759
4,No log,0.375108,0.862069
5,No log,0.377673,0.866379
6,No log,0.398011,0.857759
7,No log,0.402613,0.862069
8,No log,0.418272,0.862069
9,No log,0.423842,0.857759
10,No log,0.426379,0.857759


TrainOutput(global_step=290, training_loss=0.3554895335230334, metrics={'train_runtime': 260.0536, 'train_samples_per_second': 35.07, 'train_steps_per_second': 1.115, 'total_flos': 140611979821152.0, 'train_loss': 0.3554895335230334, 'epoch': 10.0})

In [63]:
trainer.save_state()
trainer.save_model("maghrebvibe-bert-train")

In [64]:
test = tokenizer('hadchi mazyan',truncation=True, max_length=384)

In [65]:
predictions = trainer.predict([test])
preds = np.argmax(predictions.predictions, axis=-1)


In [66]:
preds

array([1])